<a href="https://colab.research.google.com/github/SohilaOsama/eicu-code/blob/master/OurFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [609]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from scipy import stats
from scipy.stats import norm, skew, boxcox
from collections import Counter
from numpy import mean
from numpy import std
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, plot_confusion_matrix, auc
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

#XGBOOST
from xgboost import XGBClassifier

In [610]:
data = pd.read_csv("/content/data01.csv")

In [611]:
print("Data Columns --> ",data.columns)

Data Columns -->  Index(['group', 'ID', 'outcome', 'age', 'gendera', 'BMI', 'hypertensive',
       'atrialfibrillation', 'CHD with no MI', 'diabetes', 'deficiencyanemias',
       'depression', 'Hyperlipemia', 'Renal failure', 'COPD', 'heart rate',
       'Systolic blood pressure', 'Diastolic blood pressure',
       'Respiratory rate', 'temperature', 'SP O2', 'Urine output',
       'hematocrit', 'RBC', 'MCH', 'MCHC', 'MCV', 'RDW', 'Leucocyte',
       'Platelets', 'Neutrophils', 'Basophils', 'Lymphocyte', 'PT', 'INR',
       'NT-proBNP', 'Creatine kinase', 'Creatinine', 'Urea nitrogen',
       'glucose', 'Blood potassium', 'Blood sodium', 'Blood calcium',
       'Chloride', 'Anion gap', 'Magnesium ion', 'PH', 'Bicarbonate',
       'Lactic acid', 'PCO2', 'EF'],
      dtype='object')


In [612]:
data.drop(['group', 'ID'], axis = 1, inplace = True)

**Our Importance Feauters**

In [613]:
['temperature','Lymphocyte','Anion gap','Blood sodium','Lactic acid','BMI','Platelets','Creatine kinase'
,'Magnesium ion','heart rate','Leucocyte','PCO2','SP O2','Urine output','MCV', 'Urea nitrogen','Bicarbonate',
'Systolic blood pressure','RDW','Respiratory rate']

['temperature',
 'Lymphocyte',
 'Anion gap',
 'Blood sodium',
 'Lactic acid',
 'BMI',
 'Platelets',
 'Creatine kinase',
 'Magnesium ion',
 'heart rate',
 'Leucocyte',
 'PCO2',
 'SP O2',
 'Urine output',
 'MCV',
 'Urea nitrogen',
 'Bicarbonate',
 'Systolic blood pressure',
 'RDW',
 'Respiratory rate']

In [614]:
data['BMI'].fillna(data['BMI'].mean(), inplace = True)


data['heart rate'].fillna(data['heart rate'].mean(), inplace = True)
data['Systolic blood pressure'].fillna(data['Systolic blood pressure'].mean(), inplace = True)
data['Diastolic blood pressure'].fillna(data['Diastolic blood pressure'].mean(), inplace = True)
data['Respiratory rate'].fillna(data['Respiratory rate'].mean(), inplace = True)
data['SP O2'].fillna(data['SP O2'].mean(), inplace = True)
data['Urine output'].fillna(data['Urine output'].mean(), inplace = True)
data['Neutrophils'].fillna(data['Neutrophils'].mean(), inplace = True)
data['Basophils'].fillna(data['Basophils'].mean(), inplace = True)
data['Lymphocyte'].fillna(data['Lymphocyte'].mean(), inplace = True)
data['PT'].fillna(data['PT'].mean(), inplace = True)
data['INR'].fillna(data['INR'].mean(), inplace = True)
data['Creatine kinase'].fillna(data['Creatine kinase'].mean(), inplace = True)
data['glucose'].fillna(data['glucose'].mean(), inplace = True)
data['Blood calcium'].fillna(data['Blood calcium'].mean(), inplace = True)
data['PH'].fillna(data['PH'].mean(), inplace = True)
data['Lactic acid'].fillna(data['Lactic acid'].mean(), inplace = True)
data['PCO2'].fillna(data['PCO2'].mean(), inplace = True)
data['temperature'].fillna(data['temperature'].mean(), inplace = True)

print(data.isnull().sum())

outcome                     0
age                         0
gendera                     0
BMI                         0
hypertensive                0
atrialfibrillation          0
CHD with no MI              0
diabetes                    0
deficiencyanemias           0
depression                  0
Hyperlipemia                0
Renal failure               0
COPD                        0
heart rate                  0
Systolic blood pressure     0
Diastolic blood pressure    0
Respiratory rate            0
temperature                 0
SP O2                       0
Urine output                0
hematocrit                  0
RBC                         0
MCH                         0
MCHC                        0
MCV                         0
RDW                         0
Leucocyte                   0
Platelets                   0
Neutrophils                 0
Basophils                   0
Lymphocyte                  0
PT                          0
INR                         0
NT-proBNP 

In [615]:
feature_cols = ['temperature','Lymphocyte','Anion gap','Blood sodium','Lactic acid','BMI','Platelets','Creatine kinase'
,'Magnesium ion','heart rate','Leucocyte','PCO2','SP O2','Urine output','MCV', 'Urea nitrogen','Bicarbonate',
'Systolic blood pressure','RDW','Respiratory rate']

In [616]:
X = data[feature_cols] # Features
y = data.outcome

In [617]:
print(X.isnull().sum())

temperature                0
Lymphocyte                 0
Anion gap                  0
Blood sodium               0
Lactic acid                0
BMI                        0
Platelets                  0
Creatine kinase            0
Magnesium ion              0
heart rate                 0
Leucocyte                  0
PCO2                       0
SP O2                      0
Urine output               0
MCV                        0
Urea nitrogen              0
Bicarbonate                0
Systolic blood pressure    0
RDW                        0
Respiratory rate           0
dtype: int64


In [618]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1600)

In [619]:
print("X_train shape {}, len {}.".format(X_train.shape,len(X_train)))
print("X_test shape {}, len {}.".format(X_test.shape,len(X_test)))
print("Y_train shape {}, len {}.".format(y_train.shape,len(y_train)))
print("Y_test shape {}, len {}.".format(y_test.shape,len(y_test)))

X_train shape (823, 20), len 823.
X_test shape (354, 20), len 354.
Y_train shape (823,), len 823.
Y_test shape (354,), len 354.


In [620]:
print("Before Smote")
y.value_counts()

Before Smote


0    1018
1     159
Name: outcome, dtype: int64

In [621]:
from imblearn.over_sampling import SMOTE

In [622]:
sm = SMOTE(random_state=101)
X_sm, y_sm = sm.fit_resample(X, y)

In [623]:
print("After Smote")
y_sm.value_counts()

After Smote


1    1018
0    1018
Name: outcome, dtype: int64

In [624]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()


In [625]:
# fit the model with data
logreg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [626]:
y_pred=logreg.predict(X_test)

In [627]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[297,   3],
       [ 44,  10]])

In [628]:
# import required modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [629]:
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.7692307692307693
Accuracy: 0.867231638418079
Recall: 0.18518518518518517
